In [1]:
import json
import glob
import pandas as pd
import numpy as np
import pickle
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import chi2

In [2]:
filenames = glob.glob("./data/*.json")
filenames.sort()
data = []
for item in filenames:
    with open(item, 'r') as file:
        d = file.read()
    data += json.loads(d)
len(data)

7079

In [3]:
### list of important fields
# anything after 'users' is in users
fields = ['id', 'text', 'user', 'id', 'name', 'screen_name', 'verified']

In [4]:
### extract data and form a dataset.  data key is its index in the originally loaded data
keys = np.arange(len(data)).tolist()
tweet_ids, texts, name, verified = [], [], [], []
for i in range(len(data)):
    tweet_ids.append(data[i]['id'])
    texts.append(data[i]['text'])
    name.append(data[i]['user']['name'])
    verified.append(data[i]['user']['name'])

In [47]:
data[0]

{'created_at': 'Thu Sep 12 18:10:20 +0000 2019',
 'id': 1172210852415643650,
 'id_str': '1172210852415643650',
 'text': 'That’s what Americans do when others are in need – we help. We give. We inspire. Want to make a difference? There a… https://t.co/2mdo9GlaLl',
 'truncated': True,
 'entities': {'hashtags': [],
  'symbols': [],
  'user_mentions': [],
  'urls': [{'url': 'https://t.co/2mdo9GlaLl',
    'expanded_url': 'https://twitter.com/i/web/status/1172210852415643650',
    'display_url': 'twitter.com/i/web/status/1…',
    'indices': [117, 140]}]},
 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
 'in_reply_to_status_id': 1172210761583812616,
 'in_reply_to_status_id_str': '1172210761583812616',
 'in_reply_to_user_id': 813286,
 'in_reply_to_user_id_str': '813286',
 'in_reply_to_screen_name': 'BarackObama',
 'user': {'id': 813286,
  'id_str': '813286',
  'name': 'Barack Obama',
  'screen_name': 'BarackObama',
  'location': 'Washington, DC'

In [5]:
df = pd.DataFrame({'id': keys, 'tweet_id':tweet_ids,
                   'content':texts, 'author':name,
                   'verified':verified})
df.head()

,id,tweet_id,content,author,verified
0,0,1172210852415643650,That’s what Americans do when others are in ne...,Barack Obama,Barack Obama
1,1,1172210761583812616,"Jermaine Bell is just six years old, but when ...",Barack Obama,Barack Obama
2,2,1172210649654616067,Bahamian families and communities face a long ...,Barack Obama,Barack Obama
3,3,1171760194898059266,"As we remember those we lost on 9/11, we also ...",Barack Obama,Barack Obama
4,4,1171090934257397760,Marca had a remarkable way of bringing out the...,Barack Obama,Barack Obama


In [6]:
print(df.loc[178]['content'])

We are grieving with Parkland. But we are not powerless. Caring for our kids is our first job. And until we can hon… https://t.co/LWGSkzRapH


# Text cleaning and preparation

In [7]:
# \r and \n
df['content_parsed_1'] = df['content'].str.replace("\r", " ")
df['content_parsed_1'] = df['content_parsed_1'].str.replace("\n", " ")
df['content_parsed_1'] = df['content_parsed_1'].str.replace("    ", " ")

In [8]:
# " when quoting text
df['content_parsed_1'] = df['content_parsed_1'].str.replace('"', '')

In [9]:
# Lowercasing the text
df['content_parsed_2'] = df['content_parsed_1'].str.lower()

In [10]:
# punctuation sign
punctuation_signs = list("?:!.,;")
df['content_parsed_3'] = df['content_parsed_2']

for punct_sign in punctuation_signs:
    df['content_parsed_3'] = df['content_parsed_3'].str.replace(punct_sign, '')

In [11]:
# possessive nouns
df['content_parsed_4'] = df['content_parsed_3'].str.replace("'s", "")

In [12]:
# Downloading punkt and wordnet from NLTK
nltk.download('punkt')
print("------------------------------------------------------------")
nltk.download('wordnet')

------------------------------------------------------------


[nltk_data] Downloading package punkt to /Users/daisy_lab/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/daisy_lab/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [13]:
# Saving the lemmatizer into an object
wordnet_lemmatizer = WordNetLemmatizer()

In [14]:
# iterate through every word to lemmatize
nrows = len(df)
lemmatized_text_list = []

for row in range(0, nrows):
    
    # Create an empty list containing lemmatized words
    lemmatized_list = []
    
    # Save the text and its words into an object
    text = df.loc[row]['content_parsed_4']
    text_words = text.split(" ")

    # Iterate through every word to lemmatize
    for word in text_words:
        lemmatized_list.append(wordnet_lemmatizer.lemmatize(word, pos="v"))
        
    # Join the list
    lemmatized_text = " ".join(lemmatized_list)
    
    # Append to the list containing the texts
    lemmatized_text_list.append(lemmatized_text)

In [15]:
df['content_parsed_5'] = lemmatized_text_list

In [16]:
# Downloading the stop words list
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/daisy_lab/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [17]:
# Loading the stop words in english
stop_words = list(stopwords.words('english'))
stop_words[0:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [18]:
# Removing all stop words
df['content_parsed_6'] = df['content_parsed_5']

for stop_word in stop_words:

    regex_stopword = r"\b" + stop_word + r"\b"
    df['content_parsed_6'] = df['content_parsed_6'].str.replace(regex_stopword, '')

# Label coding

In [19]:
category_codes = {
    'business': 0,
    'entertainment': 1,
    'politics': 2,
    'sport': 3,
    'tech': 4
}

# TF-IDF vectors feature

In [20]:
# Parameter election
ngram_range = (1,2)
min_df = 10
max_df = 1.
max_features = 300

# preparing data
X_test = df['content_parsed_6'].tolist()

In [21]:
tfidf = TfidfVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                        stop_words=None,
                        lowercase=False,
                        max_df=max_df,
                        min_df=min_df,
                        max_features=max_features,
                        norm='l2',
                        sublinear_tf=True)

features_test = tfidf.fit_transform(X_test).toarray()
print(features_test.shape)

(7079, 300)


In [22]:
feature_names = tfidf.get_feature_names()
unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
bigrams = [v for v in feature_names if len(v.split(' ')) == 2]

In [23]:
bigrams

['actonclimate https',
 'add name',
 'climate change',
 'donald trump',
 'doyourjob https',
 'fair hear',
 'fake news',
 'governo bolsonaro',
 'health care',
 'http tco',
 'https tco',
 'judge garland',
 'live president',
 'meu deus',
 'obama speak',
 'president obama',
 'senate leaders',
 'supreme court',
 'unite state',
 'watch president',
 'years ago']

# Prediction

In [24]:
svc_path = "./models/best_svc.pickle"
tfidf_path = "./models/tfidf.pickle"
with open(svc_path, 'rb') as file:
    svc = pickle.load(file)
with open(tfidf_path, 'rb') as file:
    ft_engine = pickle.load(file)

In [35]:
feature_test = ft_engine.transform(X_test).toarray()

In [29]:
def get_category_name(category_id):
    for category, id_ in category_codes.items():    
        if id_ == category_id:
            return category

In [32]:
### making prediction
for i in range(len(X_test)):
    prediction_svc = svc.predict(feature_test[i].reshape(1, -1))[0]
    prediction_svc_proba = svc.predict_proba(feature_test[i].reshape(1, -1))

    # Return result
    category_svc = get_category_name(prediction_svc)
    print(i, "The predicted category using the SVM model is %s." %(category_svc) )
    print("The conditional probability is: %a" %(prediction_svc_proba.max()*100))
#print(prediction_svc_proba)

0 The predicted category using the SVM model is sport.
The conditional probability is: 43.30107224431667
1 The predicted category using the SVM model is sport.
The conditional probability is: 49.02145378438439
2 The predicted category using the SVM model is sport.
The conditional probability is: 80.68670049152159
3 The predicted category using the SVM model is sport.
The conditional probability is: 51.69305074157179
4 The predicted category using the SVM model is business.
The conditional probability is: 67.77086302801982
5 The predicted category using the SVM model is sport.
The conditional probability is: 77.7554766493244
6 The predicted category using the SVM model is sport.
The conditional probability is: 51.08935805851218
7 The predicted category using the SVM model is sport.
The conditional probability is: 37.32561397225625
8 The predicted category using the SVM model is sport.
The conditional probability is: 45.7544963821579
9 The predicted category using the SVM model is sport.

165 The predicted category using the SVM model is sport.
The conditional probability is: 45.98276782557641
166 The predicted category using the SVM model is sport.
The conditional probability is: 98.73290962053987
167 The predicted category using the SVM model is sport.
The conditional probability is: 37.883094867394156
168 The predicted category using the SVM model is sport.
The conditional probability is: 50.67637782875328
169 The predicted category using the SVM model is tech.
The conditional probability is: 71.04558169621365
170 The predicted category using the SVM model is sport.
The conditional probability is: 91.57502606492912
171 The predicted category using the SVM model is sport.
The conditional probability is: 90.07207211027739
172 The predicted category using the SVM model is sport.
The conditional probability is: 63.87759430882801
173 The predicted category using the SVM model is entertainment.
The conditional probability is: 99.97629199649182
174 The predicted category us

344 The predicted category using the SVM model is sport.
The conditional probability is: 37.86659674204891
345 The predicted category using the SVM model is business.
The conditional probability is: 43.8442487316205
346 The predicted category using the SVM model is sport.
The conditional probability is: 67.19983342852119
347 The predicted category using the SVM model is sport.
The conditional probability is: 98.80668672472048
348 The predicted category using the SVM model is business.
The conditional probability is: 33.450095794140154
349 The predicted category using the SVM model is sport.
The conditional probability is: 46.96804612830021
350 The predicted category using the SVM model is business.
The conditional probability is: 38.06834628865056
351 The predicted category using the SVM model is sport.
The conditional probability is: 67.19983342852119
352 The predicted category using the SVM model is politics.
The conditional probability is: 48.63755568497972
353 The predicted categor

518 The predicted category using the SVM model is sport.
The conditional probability is: 46.968526109288625
519 The predicted category using the SVM model is sport.
The conditional probability is: 84.2120087101796
520 The predicted category using the SVM model is sport.
The conditional probability is: 59.17911099073906
521 The predicted category using the SVM model is sport.
The conditional probability is: 61.751843016357476
522 The predicted category using the SVM model is business.
The conditional probability is: 47.541823488737954
523 The predicted category using the SVM model is sport.
The conditional probability is: 46.41394617081587
524 The predicted category using the SVM model is sport.
The conditional probability is: 68.6099023065008
525 The predicted category using the SVM model is sport.
The conditional probability is: 63.15130729951607
526 The predicted category using the SVM model is tech.
The conditional probability is: 37.860321835977054
527 The predicted category using 

689 The predicted category using the SVM model is entertainment.
The conditional probability is: 71.8385082468235
690 The predicted category using the SVM model is business.
The conditional probability is: 32.94995687535618
691 The predicted category using the SVM model is sport.
The conditional probability is: 61.8213283934841
692 The predicted category using the SVM model is business.
The conditional probability is: 37.40949548137873
693 The predicted category using the SVM model is sport.
The conditional probability is: 65.16592578186422
694 The predicted category using the SVM model is entertainment.
The conditional probability is: 70.33661988606814
695 The predicted category using the SVM model is business.
The conditional probability is: 38.931271321373984
696 The predicted category using the SVM model is tech.
The conditional probability is: 35.8962995532371
697 The predicted category using the SVM model is entertainment.
The conditional probability is: 77.59121275289367
698 The

...

6382 The predicted category using the SVM model is politics.
The conditional probability is: 66.27823247015363
6383 The predicted category using the SVM model is sport.
The conditional probability is: 78.50169090762556
6384 The predicted category using the SVM model is sport.
The conditional probability is: 30.788404345889276
6385 The predicted category using the SVM model is sport.
The conditional probability is: 49.917786235827336
6386 The predicted category using the SVM model is sport.
The conditional probability is: 37.86659674204891
6387 The predicted category using the SVM model is sport.
The conditional probability is: 37.86659674204891
6388 The predicted category using the SVM model is sport.
The conditional probability is: 37.86659674204891
6389 The predicted category using the SVM model is politics.
The conditional probability is: 59.899224925496455
6390 The predicted category using the SVM model is sport.
The conditional probability is: 81.57804725374767
6391 The predicted 

6546 The predicted category using the SVM model is sport.
The conditional probability is: 30.319247020875473
6547 The predicted category using the SVM model is sport.
The conditional probability is: 79.66304624628673
6548 The predicted category using the SVM model is tech.
The conditional probability is: 58.844515509202665
6549 The predicted category using the SVM model is sport.
The conditional probability is: 51.03607237111759
6550 The predicted category using the SVM model is business.
The conditional probability is: 37.224435793596385
6551 The predicted category using the SVM model is business.
The conditional probability is: 64.23835084325408
6552 The predicted category using the SVM model is business.
The conditional probability is: 70.32358239554766
6553 The predicted category using the SVM model is business.
The conditional probability is: 61.21984631930477
6554 The predicted category using the SVM model is business.
The conditional probability is: 83.75826630717408
6555 The pr

6717 The predicted category using the SVM model is sport.
The conditional probability is: 70.47589340136
6718 The predicted category using the SVM model is sport.
The conditional probability is: 78.78455538547297
6719 The predicted category using the SVM model is sport.
The conditional probability is: 37.86659674204891
6720 The predicted category using the SVM model is sport.
The conditional probability is: 38.90085379856279
6721 The predicted category using the SVM model is sport.
The conditional probability is: 63.00705741719834
6722 The predicted category using the SVM model is sport.
The conditional probability is: 74.58910310680054
6723 The predicted category using the SVM model is sport.
The conditional probability is: 53.152920104888146
6724 The predicted category using the SVM model is business.
The conditional probability is: 45.78839714984826
6725 The predicted category using the SVM model is sport.
The conditional probability is: 57.066179450138655
6726 The predicted categor

6876 The predicted category using the SVM model is business.
The conditional probability is: 53.277088075167434
6877 The predicted category using the SVM model is business.
The conditional probability is: 56.702930488392965
6878 The predicted category using the SVM model is entertainment.
The conditional probability is: 75.27475628731302
6879 The predicted category using the SVM model is sport.
The conditional probability is: 37.86659674204891
6880 The predicted category using the SVM model is politics.
The conditional probability is: 39.63012182778129
6881 The predicted category using the SVM model is sport.
The conditional probability is: 37.86659674204891
6882 The predicted category using the SVM model is entertainment.
The conditional probability is: 58.27914192515181
6883 The predicted category using the SVM model is sport.
The conditional probability is: 92.75502366802412
6884 The predicted category using the SVM model is sport.
The conditional probability is: 99.99148720025117
6

7037 The predicted category using the SVM model is sport.
The conditional probability is: 39.28305087722438
7038 The predicted category using the SVM model is sport.
The conditional probability is: 42.62757318321137
7039 The predicted category using the SVM model is sport.
The conditional probability is: 92.75502366802412
7040 The predicted category using the SVM model is sport.
The conditional probability is: 37.86659674204891
7041 The predicted category using the SVM model is sport.
The conditional probability is: 62.17961001789681
7042 The predicted category using the SVM model is business.
The conditional probability is: 54.62242481187095
7043 The predicted category using the SVM model is tech.
The conditional probability is: 45.40265170788964
7044 The predicted category using the SVM model is business.
The conditional probability is: 38.814861805395154
7045 The predicted category using the SVM model is sport.
The conditional probability is: 48.055129776977964
7046 The predicted ca

In [46]:
X_test[132]

'elections matter    power  protect  family’ health care make  plan grab  friends  vote https//tco/ry8ip1km9p'

In [48]:
texts[132]

'Elections matter. You have the power to protect a family’s health care. Make a plan, grab some friends, and vote: https://t.co/rY8Ip1km9p'